In [24]:
# ✅ 라이브러리 설치
!pip install --upgrade gspread gspread_dataframe gspread-formatting oauth2client pytz

# ✅ 모듈 불러오기
import pandas as pd
import os
from datetime import datetime
import pytz
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
import gspread
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *
from IPython.display import display, HTML

# ✅ Google Drive 마운트
drive.mount('/content/drive')

# ✅ Google Sheets 인증
json_keyfile_path = "/content/drive/My Drive/Key/credentials.json"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)
gc = gspread.authorize(credentials)

# ✅ CSV 불러오기
target_path = "/content/drive/My Drive/시설/파트너_영업권부여(영업지사,관리지사)/C150_G0000_00073.csv"
df = pd.read_csv(target_path, encoding='cp949')
df.columns = df.columns.str.strip()

# ✅ 필터링: 추천자유형 == '파트너' & 관리지사명 ≠ 영업지사명
df_filtered = df[df['추천자유형'] == "파트너"].copy()
df_filtered['지사불일치'] = df_filtered['관리지사명'] != df_filtered['영업지사명']
df_mismatch = df_filtered[df_filtered['지사불일치']]

# ✅ 필요한 컬럼 추출
columns = [
    '관리지사명', '영업지사명', '계약번호', '상호', 'KTT월정료', '서비스(중)', '서비스(소)',
    '영업자명', '영업자사번', '영업자연락처', '영업구역정보', '모집유형',
    '유통망대분류', '서비스개시일', '계약최초서비스게시일', '휴대폰'
]
df_result = df_mismatch[columns].copy()

# ✅ 계약최초서비스게시일: 뒤 6자리 제거
df_result['계약최초서비스게시일'] = df_result['계약최초서비스게시일'].astype(str).str[:8]

# ✅ 서비스(소) 공란일 경우 서비스(중)으로 대체
df_result['서비스(소)'] = df_result['서비스(소)'].where(
    df_result['서비스(소)'].astype(str).str.strip() != '',
    df_mismatch['서비스(중)']
)

# ✅ 영업자연락처: .000000 제거 + 앞에 '0' 추가
df_result['영업자연락처'] = df_result['영업자연락처'].astype(str).str.replace(r'\.0+$', '', regex=True)
df_result['영업자연락처'] = df_result['영업자연락처'].apply(lambda x: '0' + x if not x.startswith('0') else x)

# ✅ 관리지사명, 영업지사명에서 '지사' 텍스트 제거
df_result['관리지사명'] = df_result['관리지사명'].str.replace('지사', '', regex=False)
df_result['영업지사명'] = df_result['영업지사명'].str.replace('지사', '', regex=False)

# ✅ 서비스(소) 공란 → 서비스(중) 대체
df_result['서비스(소)'] = df_result['서비스(소)'].where(
    df_result['서비스(소)'].astype(str).str.strip() != '',
    df_mismatch['서비스(중)']
)

# ✅ 컬럼 순서: 영업지사명 → 관리지사명
cols = list(df_result.columns)
cols.remove('영업지사명')
cols.insert(0, '영업지사명')  # 맨 앞에 배치
df_result = df_result[cols]

# ✅ 영업자연락처 → 영업자명 옆으로 이동
cols = list(df_result.columns)
cols.remove('영업자연락처')
cols.insert(cols.index('영업자명') + 1, '영업자연락처')
df_result = df_result[cols]

# ✅ 정렬: 영업지사명 → 영업자명 내림차순
df_result = df_result.sort_values(by=['영업지사명', '영업자명'], ascending=[False, False])

# ✅ 파일명 생성 (한국시간 기준)
kst = pytz.timezone('Asia/Seoul')
now = datetime.now(kst)
date_str = now.strftime("%Y-%m-%d")
hour = now.strftime("%I").lstrip("0") or "12"
minute = now.strftime("%M")
am_pm = "오전" if now.hour < 12 else "오후"
time_str = f"{am_pm}{hour}시{minute}분"
filename = f"파트너_영업지사, 관리지사 상이한 시설_소명대상_{date_str}_{time_str}"

# ✅ CSV 저장
csv_path = f"/content/drive/My Drive/시설/파트너_영업권부여(영업지사,관리지사)/{filename}.csv"
df_result.to_csv(csv_path, index=False, encoding='cp949')
print(f"✅ CSV 저장 완료: {csv_path}")

# ✅ Google 스프레드시트 생성 및 업로드
spreadsheet = gc.create(filename)
worksheet = spreadsheet.get_worksheet(0)
set_with_dataframe(worksheet, df_result)

# ✅ 스타일 적용: 헤더 색 + 전체 셀 정렬 + 테두리
header_format = CellFormat(
    backgroundColor=Color(0, 0.2, 0.4),
    textFormat=TextFormat(bold=True, foregroundColor=Color(1, 1, 1)),
    horizontalAlignment='CENTER'
)
format_cell_range(worksheet, '1:1', header_format)

num_rows, num_cols = df_result.shape
last_col = chr(64 + num_cols)  # A~Z까지만 가능
cell_range = f"A1:{last_col}{num_rows + 1}"
body_format = CellFormat(
    borders=Borders(
        top=Border("SOLID", Color(0.8, 0.8, 0.8), 1),
        bottom=Border("SOLID", Color(0.8, 0.8, 0.8), 1),
        left=Border("SOLID", Color(0.8, 0.8, 0.8), 1),
        right=Border("SOLID", Color(0.8, 0.8, 0.8), 1)
    ),
    horizontalAlignment='CENTER'
)
format_cell_range(worksheet, cell_range, body_format)

# ✅ 결과 스프레드시트 링크 출력
sheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
print(f"✅ Google 스프레드시트 생성 완료:\n{sheet_url}")

# ✅ Google 스프레드시트 생성 및 업로드
spreadsheet = gc.create(filename)

# ✅ 공개 링크로 열람 가능하도록 설정
spreadsheet.share(None, perm_type='anyone', role='reader')

worksheet = spreadsheet.get_worksheet(0)
set_with_dataframe(worksheet, df_result)

# ✅ 모든 사용자에게 보기 권한 부여 (링크로 접속 가능하게)
spreadsheet.share(None, perm_type='anyone', role='reader')

# ✅ 보기 좋은 미리보기 (HTML + 줄바꿈 + 가로/세로 스크롤)
style = """
<style>
.preview-table {{
  border-collapse: collapse;
  width: max-content;
  min-width: 100%;
  font-family: 'Malgun Gothic', sans-serif;
  font-size: 13.5px;
  table-layout: auto;
  word-wrap: break-word;
}}
.preview-table th {{
  background-color: #003366;
  color: white;
  font-weight: bold;
  text-align: center;
  padding: 8px;
  border: 0.5px solid #ccc;
  white-space: normal;
}}
.preview-table td {{
  text-align: center;
  padding: 8px;
  border: 0.5px solid #ddd;
  white-space: normal;
  word-break: break-word;
}}
.preview-wrapper {{
  max-height: 500px;
  overflow-y: auto;
  overflow-x: auto;
  border: 1px solid #ccc;
  margin-top: 10px;
}}
</style>
<div class="preview-wrapper">
  {table}
</div>
"""
html_table = df_result.head(10).to_html(classes='preview-table', index=False, escape=False)
display(HTML(style.format(table=html_table)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ CSV 저장 완료: /content/drive/My Drive/시설/파트너_영업권부여(영업지사,관리지사)/파트너_영업지사, 관리지사 상이한 시설_소명대상_2025-05-22_오전10시55분.csv
✅ Google 스프레드시트 생성 완료:
https://docs.google.com/spreadsheets/d/1mxZXNPymSif3KWvlQ1hatlTFQz3PquG1GS2T699Q4n0


영업지사명,관리지사명,계약번호,상호,KTT월정료,서비스(중),서비스(소),영업자명,영업자연락처,영업자사번,영업구역정보,모집유형,유통망대분류,서비스개시일,계약최초서비스게시일,휴대폰
중앙,강동,52206619,대한축구협회,55000,텔레캅아이,단독형 II 지문리더,오광훈,01032280237,04637,H000405,영업직사원,에이치케이씨스템,20250509,20250509,010-6317-0174
중앙,고양,52221744,오까네 일산점,73000,텔레캅아이,GiGAeyes i-guard,남기민,01071954890,03947,G000308,텔레캅사원,마중라온,20250513,20250513,010-2301-1993
중앙,강남,52220646,리스토리성형외과 의원,100000,텔레캅아이,GiGAeyes guard+(DVR),김병조,01028891608,04251,H000103,영업직사원,에이치케이씨스템,20250509,20250509,010-4184-4201
의정부,남양주,52229821,효자동솥뚜껑-별내점(뷰),32000,텔레캅아이,ktt GiGAeyes view+(NVR),차현갑,01043250811,04571,G000602,영업직사원,건승정보통신(주),20250519,20250519,010-3063-6408
고양,남양주,52221329,옛날소고기장터국밥,45000,텔레캅아이,ktt GiGAeyes view+(NVR),성형래,01034480466,05455,G000602,영업직사원,(주)티에스시티,20250512,20250512,010-9086-2413
고양,관악,52219656,인의부부한의원,54000,텔레캅아이,GiGAeyes guard+(DVR),김현교,01079005280,05002,J000603,영업직사원,제이엔에스,20250507,20250507,010-2993-1272
고양,수원,52225065,아임스타의원,80000,GiGAeyes Basic,NaN,김현교,01079005280,05002,H000203,영업직사원,제이엔에스,20250521,20250521,010-3305-0201
강북,원주,52221151,메가오락실 단계점,110000,텔레캅아이,GiGAeyes guard+(DVR),성진수,01020805063,07864,P000201,영업직사원,(주)다온하이텍,20250502,20250502,010-5410-0420


In [ ]:
!pip install openpyxl

from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

# 엑셀 저장 경로
excel_path = f"/content/drive/My Drive/시설/파트너_영업권부여(영업지사,관리지사)/{filename}.xlsx"

# 새 워크북 및 시트 생성
wb = Workbook()
ws = wb.active
ws.title = "소명대상"

# 스타일 정의 (정렬 없음)
header_fill = PatternFill(start_color="003366", end_color="003366", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF", name="Malgun Gothic")
cell_font = Font(name="Malgun Gothic")
thin_border = Border(
    left=Side(style='thin', color="999999"),
    right=Side(style='thin', color="999999"),
    top=Side(style='thin', color="999999"),
    bottom=Side(style='thin', color="999999")
)

# 데이터 쓰기
for r_idx, row in enumerate(dataframe_to_rows(df_result, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row=r_idx, column=c_idx, value=value)
        cell.font = header_font if r_idx == 1 else cell_font
        cell.border = thin_border
        if r_idx == 1:
            cell.fill = header_fill
        # 정렬은 생략 (엑셀 기본 정렬 사용)

# 열 너비 자동 조정
for col in ws.columns:
    max_length = max(len(str(cell.value)) if cell.value else 0 for cell in col)
    adjusted_width = max_length + 2
    ws.column_dimensions[col[0].column_letter].width = adjusted_width

# 엑셀 저장
wb.save(excel_path)
print(f"✅ 줄맞춤 없는 Excel 저장 완료: {excel_path}")

✅ 줄맞춤 없는 Excel 저장 완료: /content/drive/My Drive/시설/파트너_영업권부여(영업지사,관리지사)/파트너_영업지사, 관리지사 상이한 시설_소명대상_2025-05-22_오전8시52분.xlsx
